
<a href="https://colab.research.google.com/github/aviadr1/learn-advanced-python/blob/master/content/08_test_driven_development/08-test_driven_development.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" 
     title="Open this file in Google Colab" alt="Colab"/>
</a>


# useful resources:
1. https://stackabuse.com/test-driven-development-with-pytest/
2. https://docs.pytest.org/en/latest/goodpractices.html#conventions-for-python-test-discovery
3. https://blog.ionelmc.ro/2014/05/25/python-packaging/#the-structure
4. https://github.com/vanzaj/tdd-pytest/blob/master/docs/tdd-pytest/content/tdd-basics.md



# setup

1. install `pytest`
2. install ipytest so we can run tests in the jupyter notebook

In [0]:
pip -q install ipytest pytest

In [40]:
# move to tdd directory
from pathlib import Path
if Path.cwd().name != 'tdd':
    %mkdir tdd
    %cd tdd

%pwd

'/content/tdd/tdd'

In [0]:
# cleanup all files
%rm *.py

initialize `ipytest` so it finds the tests we write in this notebook

In [0]:
import pytest
import ipytest

# enable pytest's assertions and ipytest's magics
ipytest.config(rewrite_asserts=True, magics=True)

# set the filename
__file__ = 'adv python 08 - test driven development.ipynb'



# Getting started
Lets start first by seeing the basics of pytest

# our first test

- pytests uses the following [conventions](https://docs.pytest.org/en/latest/goodpractices.html#conventions-for-python-test-discovery) to automatically discovering tests:
  1. files with tests should be called `test_*.py` or `*_test.py `
  2. test function name should start with `test_`

- to see if our code works, we can use the `assert` python keyword. pytest adds hooks to assertions to make them more useful

In [28]:
%%file test_math.py

import math
def test_add():
    assert 1+1 == 2

def test_mul():
    assert 6*7 == 42

def test_sin():
    assert math.sin(0) == 0

Writing test_math.py


now lets run pytest

In [31]:
!python -m pytest test_math.py 

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.8.1, pluggy-0.7.1
rootdir: /content, inifile:
collected 3 items                                                              

test_math.py ...                                                         [100%]

=========================== 3 passed in 0.01 seconds ===========================


Great! we just wrote 3 tests that shows that basic math still works

Hurray!

# your turn

write a test for the following function. 

if there is a bug in the function, fix it


In [1]:
%%file make_triangle.py

# version 1

def make_triangle(n):
    """
    draws a triangle using '@' letters
    for instance:
        >>> print('\n'.join(make_triangle(3))
        @
        @@
        @@@
    """

    for i in range(n):
        yield '@' * i


Writing make_triangle.py


# solution


In [19]:
%%file test_make_triangle.py

from make_triangle import make_triangle

def test_make_triangle():
    expected = "@"
    actual = '\n'.join(make_triangle(1))
    assert actual == expected

Overwriting test_make_triangle.py


In [32]:
!python -m pytest test_make_triangle.py

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.8.1, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_make_triangle.py F                                                  [100%]

=================================== FAILURES ===================================
______________________________ test_make_triangle ______________________________

    def test_make_triangle():
        expected = "@"
        actual = '\n'.join(make_triangle(1))
>       assert actual == expected
E       AssertionError: assert '' == '@'
E         + @

test_make_triangle.py:7: AssertionError
=========================== 1 failed in 0.02 seconds ===========================


so the expected is `'@'` and the actual is `''` ...

this is a bug! lets fix the code and re-run

In [33]:
%%file make_triangle.py

# version 2 
def make_triangle(n):
    """
    draws a triangle using '@' letters
    for instance:
        >>> print('\n'.join(make_triangle(3))
        @
        @@
        @@@
    """

    for i in range(1, n+1):
        yield '@' * i

Overwriting make_triangle.py


In [34]:
!python -m pytest test_make_triangle.py

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.8.1, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_make_triangle.py .                                                  [100%]

=========================== 1 passed in 0.01 seconds ===========================


# Using fixtures to simplify tests



## Motivating example

Lets look at an example of class `Person`, where each person has a name and remembers their friends.

In [5]:
%%file person.py

#version 1
class Person:
    def __init__(self, name, favorite_color, year_born):
        self.name = name
        self.favorite_color = favorite_color
        self.year_born = year_born
        self.friends = set()

    def add_friend(self, other_person):
        self.friends.add(other_person)
        other_person.friends.add(self)

    def __repr__(self):
        return f'Person(name={self.name!r}, '  \
               f'favorite_color={self.favorite_color!r}, ' \
               f'year_born={self.year_born!r}, ' \
               f'friends={[f.name for f in self.friends]})'


Writing person.py


Lets write a test for `add_friend()` function.

notice how the setup for the test is taking so much of the function, while also requiring _inventing_ a lot of repetitious data

is there a way to reduce this boiler plate code

In [83]:
%%file test_person.py

from person import Person

def test_name():
    # setup
    terry = Person(
        'Terry Gilliam',
        'red',
        1940
        )
    
    # test
    assert terry.name == 'Terry Gilliam' 


def test_add_friend():
    # setup for the test 
    terry = Person(
        'Terry Gilliam',
        'red',
        1940
        )
    eric = Person(
        'Eric Idle',
        'blue',
        1943
        )
    
    # actual test
    terry.add_friend(eric)
    assert eric in terry.friends
    assert terry in eric.friends

Overwriting test_person.py


In [85]:
!python -m pytest -q test_person.py

..                                                                       [100%]
2 passed in 0.01 seconds


## Fixtures to the rescue




what is we had a magic factory that can conjure up a name, favorite color and birth year?

then we could write our `test_name()` more simply like this:

```python
def test_name(person_name, favorite_color, birth_year):
    person = Person(person_name, favorite_color, birth_year)
    
    # test
    assert person.name == person_name 
```


furthermore, if we had a magic factory that can create `terry` and `eric` we could write our `test_add_friend()` function like this:

```python
def test_add_friend(eric, terry):
    eric.add_friend(terry)
    assert eric in terry.friends
    assert terry in eric.friends
```


fixtures in `pytest` allow us to create such magic factories using the `@pytest.fixture` notation.

here's an example:

In [6]:
%%file test_person_fixtures1.py

import pytest
from person import Person

@pytest.fixture
def person_name():
    return 'Terry Gilliam'

@pytest.fixture
def birth_year():
    return 1940

@pytest.fixture
def favorite_color():
    return 'red'

def test_person_name(person_name, favorite_color, birth_year):
    person = Person(person_name, favorite_color, birth_year)
 
    # test
    assert person.name == person_name 

Overwriting test_person_fixtures1.py


In [7]:
!python -m pytest test_person_fixtures1.py

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.8.1, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_person_fixtures1.py .                                               [100%]

=========================== 1 passed in 0.02 seconds ===========================


what's happening here?

`pytest` sees that the test function `test_person_name(person_name, favorite_color, birth_year)` requires three parameters, and searches for fixtures annotated with `@pytest.fixture` with the same name.

when it finds them, it calls these fixtures on our behalf, and passes the return value as the parameter. in effect, it calls

```python
test_person_name(person_name=person_name(), favorite_color=favorite_color(), birth_year=birth_year()
```

note how much code this saves

# your turn
1. rewrite the `test_add_friend` function to accept two parameters `def test_add_friend(eric, terry)` 
2. write fixtures for eric and terry
3. run pytest

# solution


In [8]:
%%file test_person_fixtures2.py

import pytest
from person import Person

@pytest.fixture
def eric():
    return Person('Eric Idle', 'red', 1943)

@pytest.fixture
def terry():
    return Person('Terry Gilliam', 'blue', 1940)

def test_add_friend(eric, terry):
    eric.add_friend(terry)
    assert eric in terry.friends
    assert terry in eric.friends
    

Writing test_person_fixtures2.py


In [9]:
!python -m pytest -q test_person_fixtures2.py

.                                                                        [100%]
1 passed in 0.02 seconds


# Codebase to test: class Person

Lets reuse the `Person` and `OlympicRunner` classes we've defined in earlier chapters in order to see how to write tests


In [26]:
%%file person.py

# Person v1
class Person:
    def __init__(self, name):
        name = name
    def __repr__(self):
        return f"{type(self).__name__}({self.name!r})"
    def walk(self):
        print(self.name, 'walking')
    def run(self):
        print(self.name,'running')
    def swim(self):
        print(self.name,'swimming')
        
class OlympicRunner(Person):
    def run(self):
        print(self.name,self.name,"running incredibly fast!")
        
    def show_medals(self):
        print(self.name, 'showing my olympic medals')
    
def train(person):
    person.walk()
    person.swim()
    person.run()

Overwriting person.py


# our first test

- [conventions](https://docs.pytest.org/en/latest/goodpractices.html#conventions-for-python-test-discovery) 
  1. files with tests should be called `test_*.py` or `*_test.py `
  2. test function name should start with `test_`

- to see if our code works, we can use the `assert` python keyword. pytest adds hooks to assertions to make them more useful

In [31]:
%%file test_person1.py
from person import Person

# our first test
def test_preson_name():
    terry = Person('Terry Gilliam')
    assert terry.name == 'Terry Gilliam'

Overwriting test_person1.py


In [32]:
!python -m pytest

============================= test session starts ==============================
platform linux -- Python 3.6.9, pytest-3.6.4, py-1.8.1, pluggy-0.7.1
rootdir: /content, inifile:
collected 1 item                                                               

test_person1.py F                                                        [100%]

=================================== FAILURES ===================================
_______________________________ test_preson_name _______________________________

    def test_preson_name():
        terry = Person('Terry Gilliam')
>       assert terry.name == 'Terry Gilliam'
E       AttributeError: 'Person' object has no attribute 'name'

test_person1.py:6: AttributeError
=========================== 1 failed in 0.03 seconds ===========================


## lets run our tests


In [0]:
# execute the tests via pytest, arguments are passed to pytest
ipytest.run('-qq')

ERROR: file not found: adv python 08 - test driven development.ipynb



## running our first test


In [0]:
# very simple test
def test_person_repr1():
    assert str(Person('terry gilliam')) == f"Person('terry gilliam')"

# test using mock object
def test_train1():
    person = mocking.Mock()
    
    train(person)
    person.walk.assert_called_once()
    person.run.assert_called_once()
    person.swim.assert_called_once()

# create factory for person's name
@pytest.fixture
def person_name():
    return 'terry gilliam'
    
# create factory for Person, that requires a person_name 
@pytest.fixture
def person(person_name):
    return Person(person_name)

# test using mock object
def test_train2(person):
    # this makes sure no other method is called
    person = mocking.create_autospec(person)
    
    train(person)
    person.walk.assert_called_once()
    person.run.assert_called_once()
    person.swim.assert_called_once()


# test Person using and request a person, person_name from the fixtures
def test_person_repr2(person, person_name):
    assert str(person) == f"Person('{person_name}')"
    
# fixture with multiple values
@pytest.fixture(params=['usain bolt', 'Matthew Wells'])
def olympic_runner_name(request):
    return request.param

@pytest.fixture
def olympic_runner(olympic_runner_name):
    return OlympicRunner(olympic_runner_name)

# test train() using mock object for print
@mocking.patch('builtins.print')
def test_train3(mocked_print, olympic_runner):
    train(olympic_runner)
    mocked_print.assert_called()

In [0]:
# execute the tests via pytest, arguments are passed to pytest
ipytest.run('-qq')

......                                                                                                           [100%]
